In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from scripts.compute_kpis import latency_ms

## Import Data

In [ ]:
df = pd.read_csv('../data/kpis.csv', parse_dates=['timestamp'])
df.head()

## Data Sanity Checks

In [ ]:
df.isna().sum()
df[['latency_ms', 'link_qulaity', 'handover']].describe()

## Time Series of Latency & Link Quality

In [ ]:
fig = px.line(df, x='timestamp', y='latency_ms', title='Satellite RTT(ms) Over Time', labels={'latency_ms':'RTT (ms)'})
fig.show()

fig2 = px.line(df, x='timestamp', y='link_qulaity', title='Link Quality Over Time', labels={'link_quality':'Link Quality (1/km)'})
fig2.show()

## Handover Events

In [ ]:
df['cum_handover'] = df['handover'].cumsum()
fig3 = px.line(df, x='timestamp', y='cum_handover', title='Cumulative Handover Count')
fig3.show()

handover_counts = df[df['handover'] == 1]['closest_gs'].value_counts().reset_index()
handover_counts.columns = ['ground_station', 'handover_events']
fig4 = px.bar(handover_counts, x='ground_station', y='handover_events', title='Handover Events per Ground Stations')
fig4.show()

## Distribution and Correlation

In [ ]:
fig5 = px.histogram(df, x='latency_ms', nbins=30, marginal='box', title='Latency Distribution')
fig5.show()

fig6 = px.scatter(df, x='link_qulaity', y='latency_ms', title='Latency vs Link Quality', labels={ 'link_qulaity': 'Link Quality', 'latency_ms': 'RTT (ms)' })
fig6.show()

## 3D orbit plot

In [ ]:
df_positions = pd.read_csv('../data/simulated_passes.csv', parse_dates=['timestamp'])
df_kpis = pd.read_csv('../data/kpis.csv', parse_dates=['timestamp'])

df_merged = pd.merge(df_positions, df_kpis, on='timestamp')

fig7 = px.scatter_3d(df_merged, x='x_km', y='y_km', z='z_km',
                     color='latency_ms',
                     title='3D Orbit Colored by RTT (ms)',
                     labels={'latency_ms':'RTT (ms)'})
fig7.update_traces(marker=dict(size=3))
fig7.show()